In [4]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import dump
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, train_test_split

In [5]:
model_preds = dump.load('book_recommender.py')

In [6]:
df = pd.read_csv('filtered_df.csv')
df.head()

,isbn,likes,rating,title,user_id,user_name,author,mean_rating,num_ratings,num_reviews
0,0399562451,390,3,Still Me,9684061,Melissa,Jojo Moyes,4.17,96193,9442
1,0399562451,309,3,Still Me,5032725,Debra,Jojo Moyes,4.17,96193,9442
2,0399562451,208,5,Still Me,665584,Alejandro,Jojo Moyes,4.17,96193,9442
3,0399562451,189,5,Still Me,57991562,Pauline,Jojo Moyes,4.17,96193,9442
4,0399562451,169,4,Still Me,5872506,Larry H,Jojo Moyes,4.17,96193,9442


In [8]:
model = SVD(n_epochs=17, lr_all=.015, reg_all=.125, n_factors=17)

In [9]:
def create_user():
    user_name = input('Please enter your name: ')
    print("\n Welcome {}! I'm going to recommend some books just for you.\n".format(user_name))
    user_id = max(df.user_id) + 1
    return user_name, user_id

In [10]:
new_user_name, new_user_id = create_user()

Please enter your name: dmillz

 Welcome dmillz! I'm going to recommend some books just for you.



In [8]:
def valid_entries(user_entry, valid_entries):
    if user_entry not in valid_entries:
        return False
    else:
        return True

In [9]:
def check_favorite_authors(df):
    valid_yes_no = ['Y', 'N', 'YES', 'NO']
    is_valid = False
    while is_valid == False:
        favorite_author_ask = (input("Do you have any favorite authors who may have recently written books? (Yes/No) ")).upper()
        if valid_entries(favorite_author_ask, valid_yes_no) == False:
            print("Sorry, that isn't a valid entry, please try again. \n")
        else:
            is_valid = True
            if favorite_author_ask in ['Y', 'YES']:
                favorite_author_ask = True
            else:
                favorite_author_ask = False
    if favorite_author_ask == True:
        keep_checking = True
        while keep_checking == True:
            favorite_author = (input("Name an author, I will check to see if they wrote anything on the top list."))
            if favorite_author.title() in df.author.unique():
                print("{} has written the following books: \n".format(favorite_author))
                for i in list(df.title.loc[df.author == favorite_author].unique()):
                    print(i + "\n")
            else:
                print("Sorry, {} has not written any books that were recently trending on our lists.")
            is_valid = False
            while is_valid == False:
                check_again = input("Would you like to check another author? (Yes/No)").upper()
                if valid_entries(check_again, valid_yes_no) == False:
                    print("Sorry, that isn't a valid entry, please try again. \n")
                else:
                    is_valid = True
            if check_again in ['Y', 'Yes']:
                keep_checking = True
            else:
                keep_checking = False

In [ ]:
check_favorite_authors(df)

Do you have any favorite authors who may have recently written books? (Yes/No) Yes
Name an author, I will check to see if they wrote anything on the top list.Chrissy Teigen
Chrissy Teigen has written the following books: 

Cravings: Hungry for More



In [14]:
def num_reviews():
    x = False
    while x == False:
        num_reviews = input('How many books would you like to review? \n')
        try:
            if 0< int(num_reviews) <=10:
                x = True
                return int(num_reviews)
                break
        except:
            print('Invalid entry. Please try again.')

In [15]:
num_reviews = num_reviews()

How many books would you like to review? 
5


In [118]:
num_reviews

5

In [30]:
df.head()

,isbn,likes,rating,title,user_id,user_name,author,mean_rating,num_ratings,num_reviews
0,0399562451,390,3,Still Me,9684061,Melissa,Jojo Moyes,4.17,96193,9442
1,0399562451,309,3,Still Me,5032725,Debra,Jojo Moyes,4.17,96193,9442
2,0399562451,208,5,Still Me,665584,Alejandro,Jojo Moyes,4.17,96193,9442
3,0399562451,189,5,Still Me,57991562,Pauline,Jojo Moyes,4.17,96193,9442
4,0399562451,169,4,Still Me,5872506,Larry H,Jojo Moyes,4.17,96193,9442


In [16]:
book_info_df = df[['title','author', 'isbn']].drop_duplicates().reset_index(drop=True)

In [17]:
def new_user_ratings(book_info_df, num_reviews, user_name, user_id):
    user_id = new_user_id
    user_name = new_user_name
    ratings_list = []
    while num_reviews >0: 
        book = book_info_df.sample(1)
        rating = input('How do you rate this book on a scale of 1 to 5? If you are unfamiliar with this title press n. \n')
        print(book.title.item() + ' by ' + book.author.item())
        if rating == 'n':
            continue
        else:
            rating_dict = {'user_id':user_id, 'isbn':book.isbn.item(), 'rating':rating}
            ratings_list.append(rating_dict)
            num_reviews -= 1
    return ratings_list
        

In [18]:
new_ratings = new_user_ratings(book_info_df, num_reviews, new_user_name, new_user_id)

How do you rate this book on a scale of 1 to 5? If you are unfamiliar with this title press n. 
4
All-American Murder: The Rise and Fall of Aaron Hernandez, the Superstar Whose Life Ended on Murderers' Row by James Patterson
How do you rate this book on a scale of 1 to 5? If you are unfamiliar with this title press n. 
4
Alphas Like Us by Krista Ritchie
How do you rate this book on a scale of 1 to 5? If you are unfamiliar with this title press n. 
4
My Year of Rest and Relaxation by Ottessa Moshfegh
How do you rate this book on a scale of 1 to 5? If you are unfamiliar with this title press n. 
4
The Death of Mrs. Westaway by Ruth Ware
How do you rate this book on a scale of 1 to 5? If you are unfamiliar with this title press n. 
4
Iron and Magic by Ilona Andrews


In [20]:
new_ratings

[{'user_id': 92934394, 'isbn': '0316412651', 'rating': '4'},
 {'user_id': 92934394, 'isbn': '163576517X', 'rating': '4'},
 {'user_id': 92934394, 'isbn': '1787330419', 'rating': '4'},
 {'user_id': 92934394, 'isbn': '1501156217', 'rating': '4'},
 {'user_id': 92934394, 'isbn': '1641970405', 'rating': '4'}]

In [21]:
pd.concat([df[['user_id', 'isbn', 'rating']],pd.DataFrame(new_ratings)[['user_id', 'isbn', 'rating']]]).reset_index(drop=True)

,user_id,isbn,rating
0,9684061,0399562451,3
1,5032725,0399562451,3
2,665584,0399562451,5
3,57991562,0399562451,5
4,5872506,0399562451,4
5,16254355,0399562451,4
6,7365405,0399562451,4
7,25760543,0399562451,1
8,14002983,0399562451,4
9,4514828,0399562451,4


In [22]:
df.head()

,isbn,likes,rating,title,user_id,user_name,author,mean_rating,num_ratings,num_reviews
0,0399562451,390,3,Still Me,9684061,Melissa,Jojo Moyes,4.17,96193,9442
1,0399562451,309,3,Still Me,5032725,Debra,Jojo Moyes,4.17,96193,9442
2,0399562451,208,5,Still Me,665584,Alejandro,Jojo Moyes,4.17,96193,9442
3,0399562451,189,5,Still Me,57991562,Pauline,Jojo Moyes,4.17,96193,9442
4,0399562451,169,4,Still Me,5872506,Larry H,Jojo Moyes,4.17,96193,9442


In [24]:
def new_recommendations(df, new_ratings):
    df = df[['user_id', 'isbn', 'rating']]
    new_ratings = pd.DataFrame(new_ratings)[['user_id', 'isbn', 'rating']]
    new_df = pd.concat([df, new_ratings]).reset_index(drop=True)
    reader = Reader(rating_scale=(1,5))
    data = Dataset.load_from_df(new_df,reader)
    train, test = train_test_split(data, test_size=.2)
    model = SVD(n_epochs=17, lr_all=.015, reg_all=.125, n_factors=17)
    model.fit(train)
    preds = model.test(test)
    user_id = new_ratings.user_id[0]
    book_list = []
    for x in new_df.isbn.unique():
        book_list.append((x, model.predict(user_id,x)[3]))
    ranked_books = sorted(book_list, key=lambda x: x[1], reverse=True)
    return ranked_books

In [33]:
recs = new_recommendations(df, new_ratings)
recs[0][0]

'1524759724'

In [26]:
df[['user_id', 'isbn', 'rating']]

,user_id,isbn,rating
0,9684061,0399562451,3
1,5032725,0399562451,3
2,665584,0399562451,5
3,57991562,0399562451,5
4,5872506,0399562451,4
5,16254355,0399562451,4
6,7365405,0399562451,4
7,25760543,0399562451,1
8,14002983,0399562451,4
9,4514828,0399562451,4


In [27]:
df.head()

,isbn,likes,rating,title,user_id,user_name,author,mean_rating,num_ratings,num_reviews
0,0399562451,390,3,Still Me,9684061,Melissa,Jojo Moyes,4.17,96193,9442
1,0399562451,309,3,Still Me,5032725,Debra,Jojo Moyes,4.17,96193,9442
2,0399562451,208,5,Still Me,665584,Alejandro,Jojo Moyes,4.17,96193,9442
3,0399562451,189,5,Still Me,57991562,Pauline,Jojo Moyes,4.17,96193,9442
4,0399562451,169,4,Still Me,5872506,Larry H,Jojo Moyes,4.17,96193,9442


In [36]:
df_reduced = df[['title', 'author', 'isbn']].drop_duplicates()
df_reduced.loc[df_reduced.isbn == recs[0][0]].title.item()

'Cravings: Hungry for More'

In [29]:
def num_recs():
    x = False
    while x == False:
        num_recs = input('How many books would you like to be recommended? \n')
        try:
            if 0< int(num_recs) <=10:
                x = True
                return int(num_recs)
                break
        except:
            print('Invalid entry. Please try again.')
    return num_recs

In [30]:
num_recs = num_recs()

How many books would you like to be recommended? 
5


In [31]:
num_recs

5

In [41]:
def recs_to_return(recs, df, num_recs):
    df_reduced = df[['title', 'author', 'isbn']].drop_duplicates()
    for i, v in enumerate(recs):
        book = df_reduced.loc[df_reduced.isbn == v[0]]
        print('Recommendation # ' + str(i+1) + ': ' + book.title.item() + ' by ' + book.author.item() + 
              ' \n https://isbnsearch.org/isbn/{}'.format(book.isbn.item())) 
        num_recs-=1
        if num_recs == 0:
            break

In [42]:
recs_to_return(recs, df, num_recs)

Recommendation # 1: Cravings: Hungry for More by Chrissy Teigen 
 https://isbnsearch.org/isbn/1524759724
Recommendation # 2: Becoming by Michelle Obama 
 https://isbnsearch.org/isbn/1524763136
Recommendation # 3: The Help by Kathryn Stockett 
 https://isbnsearch.org/isbn/0425232204
Recommendation # 4: War of the Wolf by Bernard Cornwell 
 https://isbnsearch.org/isbn/0062563173
Recommendation # 5: A Day in the Life of Marlon Bundo by Jill Twiss 
 https://isbnsearch.org/isbn/145217380X


In [99]:
y = pd.DataFrame(new_ratings)[['user_id', 'isbn', 'rating']]
y.user_id[0]

92934394